# Calculate commute distance between home - office

In [8]:
import arcpy
from arcpy import env
import os

try:
    
    #The NA layer's data will be saved to the workspace specified here
    env.workspace = r"C:\Users\kenta\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb"
    env.overwriteOutput = True

    #Set local variables
    input_gdb1 = r"C:\Users\kenta\Documents\GISData_Import\Esri道路2014中国\道路網_鳥取県_島根県_岡山県_広島県_山口県版\ADF2014.gdb"
    input_gdb2 = r"C:\Users\kenta\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb"
    input_gdb3 = r"C:\Users\kenta\Documents\ArcGIS\Projects\HealthPlaceNew\E2FCA.gdb"
    network = os.path.join(input_gdb1, "ADF", "ADF_ND")
    stops_home = os.path.join(input_gdb3, "dfnew6Gecoded")
    stops_work = os.path.join(input_gdb2, "WorkAddress2015_Geocode")
    layer_name = "CommuteDistance"
    out_routes_featureclass = "CommuteDistance"
    travel_mode = "Driving Time"

    #Create a new Route layer.  Optimize on driving time, but compute the
    #distance traveled by accumulating the Meters attribute.
    result_object = arcpy.na.MakeRouteAnalysisLayer(network, layer_name, travel_mode)

    #Get the layer object from the result object. 
    #The route layer can now be referenced using the layer object.
    layer_object = result_object.getOutput(0)

    #Get the names of all the sublayers within the route layer.
    sublayer_names = arcpy.na.GetNAClassNames(layer_object)
    #Stores the layer names that we will use later
    stops_layer_name = sublayer_names["Stops"]
    routes_layer_name = sublayer_names["Routes"]

    #Before loading the commuters' home and work locations as route stops, set
    #up field mapping.  Map the "ID" field from the input data to
    #the RouteName property in the Stops sublayer, which ensures that each
    #unique ID will be placed in a separate route.  Matching
    #ID from home and office will end up in the same route.
    field_mappings = arcpy.na.NAClassFieldMappings(layer_object, stops_layer_name)
    field_mappings["RouteName"].mappedFieldName = "ID"

    #Add the commuters' home and work locations as Stops. The same field mapping
    #works for both input feature classes because they both have a field called
    #"Commuter_Name"
    arcpy.na.AddLocations(layer_object, stops_layer_name, stops_home,
                        field_mappings, "")
    arcpy.na.AddLocations(layer_object, stops_layer_name, stops_work,
                        field_mappings, "", append="APPEND")

    #Solve the route layer.
    arcpy.na.Solve(layer_object)

    # Get the output Routes sublayer and save it to a feature class
    routes_sublayer = layer_object.listLayers(routes_layer_name)[0]
    arcpy.management.CopyFeatures(routes_sublayer, out_routes_featureclass)

    print("Script completed successfully")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully
